In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
!pip install keras-tuner

In [ ]:
import keras_tuner as kt

In [ ]:
(img_train, label_train), (img_test, label_test) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [ ]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer[32-512]
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(tf.keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(tf.keras.layers.Dense(10))

  # Tune the learning rate for optimizer
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate), 
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=['accuracy']
  )

  return model


In [ ]:
tuner = kt.Hyperband(
    model_builder, 
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='my_dir',
    project_name='intro_to_kt'
)

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
tuner.search(
    img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early]
)

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
print('best combination: units={}, lr={}'.format(best_hps.get('units'), best_hps.get('learning_rate')))

In [ ]:
model = tuner.hypermodel.build(best_hps)

In [ ]:
history = model.fit(
  img_train, label_train, epochs=10, validation_split=0.2
)

In [ ]:
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print(best_epoch)

In [ ]:
eval_result = model.evaluate(img_test, label_test)
print('[test_loss, test_accuracy]:', eval_result)